<a href="https://colab.research.google.com/github/fatmas1982/Neural-network-implementation-from-scratch/blob/main/Chapter_5_Calculating_Network_Error_with_Loss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.youtube.com/watch?v=hbt4j9XDVnA 
أساس العمليات الحسابية وكيف وصلنا عن طريقها للوغاريتمات؟

الاعتلاج او الانتروبي في علم البيانات 
https://www.youtube.com/watch?v=hVsvjvtKM7I


https://www.bel3arabi.me/ما-هو-الإعتلاج-في-علم-البيانات/


#**Categorical Cross-Entropy Loss**

In [ ]:
import numpy as np

In [ ]:
softmax_output=np.array([[.7,.1,.2],
          [.1,.5,.4],
          [.02,.9,.08]])

In [ ]:
class_target=[0,1,1]

In [ ]:
#r=zip(class_target,softmax_output)

In [ ]:
for target_idx,distrupution in zip(class_target,softmax_output):
    print("target_idx",target_idx,"distrupution",distrupution,"results",distrupution[target_idx])
    print("***********************")

target_idx 0 distrupution [0.7 0.1 0.2] results 0.7
***********************
target_idx 1 distrupution [0.1 0.5 0.4] results 0.5
***********************
target_idx 1 distrupution [0.02 0.9  0.08] results 0.9
***********************


In [ ]:
distrupution

array([0.02, 0.9 , 0.08])

In [ ]:
print(softmax_output[[0,1,2],class_target])

[0.7 0.5 0.9]


In [ ]:
range(len(softmax_output))

range(0, 3)

In [ ]:

print(softmax_output[range(len(softmax_output)),class_target])

[0.7 0.5 0.9]


In [ ]:
neg_log=-np.log(softmax_output[[0,1,2],class_target])

In [ ]:
neg_log

array([0.35667494, 0.69314718, 0.10536052])

In [ ]:
print(-np.log(softmax_output[range(len(softmax_output)),class_target]))

[0.35667494 0.69314718 0.10536052]


In [ ]:
np.mean(neg_log)

0.38506088005216804

In [ ]:
(0.35667494+ 0.69314718+0.10536052)/3

0.38506088

In [ ]:
class_targets=np.array([[1,0,0],
          [0,1,0],
          [0,1,0]])

In [ ]:
class_targets.shape

(3, 3)

In [ ]:
len(class_targets.shape)

2

In [ ]:
if len(class_targets.shape)==1:
    correct_confidance=softmax_output[range(len(softmax_output)),class_target]
elif len(class_targets.shape)==2:
    correct_confidance=np.sum(softmax_output*class_target,axis=1)


In [ ]:
correct_confidance

array([0.3 , 0.9 , 0.98])

In [ ]:
neg_log=-np.log(correct_confidance)

In [ ]:
avg_loss=np.mean(neg_log)

In [ ]:
avg_loss

0.4431786757670939

In [ ]:
np.log(0)

<ipython-input-66-f6e7c0610b57>:1: RuntimeWarning: divide by zero encountered in log
  np.log(0)


-inf

#**The Categorical Cross-Entropy Loss Class**

In [ ]:
!pip install nnfs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import nnfs
from nnfs.datasets import spiral_data

nnfs.init()




In [ ]:
# Dense layer
class Layer_Dense:

    # Layer initialization
    def __init__(self, n_inputs, n_neurons):
        # Initialize weights and biases
        self.weights = 0.01 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))

    # Forward pass
    def forward(self, inputs):
        # Calculate output values from inputs, weights and biases
        self.output = np.dot(inputs, self.weights) + self.biases






In [ ]:
# ReLU activation
class Activation_ReLU:

    # Forward pass
    def forward(self, inputs):
        # Calculate output values from inputs
        self.output = np.maximum(0, inputs)



In [ ]:
# Softmax activation
class Activation_Softmax:

    # Forward pass
    def forward(self, inputs):

        # Get unnormalized probabilities
        exp_values = np.exp(inputs - np.max(inputs, axis=1,
                                            keepdims=True))
        # Normalize them for each sample
        probabilities = exp_values / np.sum(exp_values, axis=1,
                                            keepdims=True)

        self.output = probabilities




In [ ]:
# Common loss class
class Loss:

    # Calculates the data and regularization losses
    # given model output and ground truth values
    def calculate(self, output, y):

        # Calculate sample losses
        sample_losses = self.forward(output, y)

        # Calculate mean loss
        data_loss = np.mean(sample_losses)

        # Return loss
        return data_loss




In [ ]:
# Cross-entropy loss
class Loss_CategoricalCrossentropy(Loss):

    # Forward pass
    def forward(self, y_pred, y_true):

        # Number of samples in a batch
        samples = len(y_pred)

        # Clip data to prevent division by 0
        # Clip both sides to not drag mean towards any value
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)
        print(len(y_pred_clipped))

        # Probabilities for target values -
        # only if categorical labels
        print(y_true.shape)
        print (len(y_true.shape))
        if len(y_true.shape) == 1:
            correct_confidences = y_pred_clipped[
                range(samples),
                y_true
            ]

        # Mask values - only for one-hot encoded labels
        elif len(y_true.shape) == 2:
            correct_confidences = np.sum(
                y_pred_clipped*y_true,
                axis=1
            )

        # Losses
        negative_log_likelihoods = -np.log(correct_confidences)
        return negative_log_likelihoods





In [ ]:
# Create dataset
X, y = spiral_data(samples=100, classes=3)

# Create Dense layer with 2 input features and 3 output values
dense1 = Layer_Dense(2, 3)

# Create ReLU activation (to be used with Dense layer):
activation1 = Activation_ReLU()

# Create second Dense layer with 3 input features (as we take output
# of previous layer here) and 3 output values
dense2 = Layer_Dense(3, 3)

# Create Softmax activation (to be used with Dense layer):
activation2 = Activation_Softmax()

# Create loss function
loss_function = Loss_CategoricalCrossentropy()

# Perform a forward pass of our training data through this layer
dense1.forward(X)

# Perform a forward pass through activation function
# it takes the output of first dense layer here
activation1.forward(dense1.output)


# Perform a forward pass through second Dense layer
# it takes outputs of activation function of first layer as inputs
dense2.forward(activation1.output)

# Perform a forward pass through activation function
# it takes the output of second dense layer here
activation2.forward(dense2.output)

# Let's see output of the first few samples:
print(activation2.output[:5])

# Perform a forward pass through loss function
# it takes the output of second dense layer here and returns loss
loss = loss_function.calculate(activation2.output, y)

# Print loss value
print('loss:', loss)

[[0.33333334 0.33333334 0.33333334]
 [0.33333334 0.33333334 0.33333334]
 [0.33333328 0.3333334  0.3333333 ]
 [0.3333333  0.33333334 0.3333333 ]
 [0.33333334 0.33333334 0.33333334]]
300
(300,)
1
loss: 1.0986141


NameError: ignored

In [ ]:
len(y.shape)

1

In [ ]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], d

In [ ]:
predictions= np.argmax(activation2.output,axis=1)
if len(y.shape) ==2:
    y=np.argmax(y,axis=1)
    
accuracy=np.mean(predictions==y)
# Print accuracy
print('acc:',accuracy)


acc: 0.34


In [ ]:
predictions==y

array([ True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
       False, False, False,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,